In [30]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Constants
feature_vector_size = 12544
num_samples = 300  # Adjust as needed
sequence_length = 10  # Number of timesteps in each sequence

In [31]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2  # Reserve 20% for validation
)


In [32]:
train_generator = datagen.flow_from_directory(
    'Data/AugmentedAlzheimerDataset',
    target_size=(64, 64),
    batch_size=1,
    class_mode='categorical',
    subset='training'  # Specify training subset
)

validation_generator = datagen.flow_from_directory(
    'Data/AugmentedAlzheimerDataset',
    target_size=(64, 64),
    batch_size=1,
    class_mode='categorical',
    subset='validation'  # Specify validation subset
)

Found 27188 images belonging to 4 classes.
Found 6796 images belonging to 4 classes.


In [33]:
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten()
])

In [34]:
train_sequences = np.zeros((num_samples, sequence_length, feature_vector_size))
train_labels = []

for i in range(num_samples):
    sequence_labels = []
    for j in range(sequence_length):
        img, label = next(train_generator)
        feature_vector = cnn_model.predict(img)  # Extract features
        train_sequences[i, j, :] = feature_vector.flatten()
    train_labels.append(label[0])  # Append the one-hot encoded label

# Convert training labels to numpy array
train_labels = np.array(train_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━

In [35]:

validation_sequences = np.zeros((num_samples, sequence_length, feature_vector_size))
validation_labels = []

for i in range(num_samples):
    sequence_labels = []
    for j in range(sequence_length):
        img, label = next(validation_generator)
        feature_vector = cnn_model.predict(img)  # Extract features
        validation_sequences[i, j, :] = feature_vector.flatten()
    validation_labels.append(label[0])  # Append the one-hot encoded label

# Convert validation labels to numpy array
validation_labels = np.array(validation_labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━

In [36]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(sequence_length, feature_vector_size)))
model.add(Dense(4, activation='softmax'))  # 4 classes

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
model.fit(
    train_sequences,
    train_labels,
    epochs=10,
    batch_size=16,
    validation_data=(validation_sequences, validation_labels)
)

Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.2407 - loss: 1.6298 - val_accuracy: 0.2900 - val_loss: 1.3851
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2936 - loss: 1.3848 - val_accuracy: 0.2833 - val_loss: 1.3824
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.2620 - loss: 1.3814 - val_accuracy: 0.2833 - val_loss: 1.3794
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3238 - loss: 1.3831 - val_accuracy: 0.2933 - val_loss: 1.3822
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3090 - loss: 1.3833 - val_accuracy: 0.2933 - val_loss: 1.3810
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.2942 - loss: 1.3822 - val_accuracy: 0.2833 - val_loss: 1.3786
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.2950 - loss: 1.3724 - val_accuracy: 0.2900 - val_loss: 1.3790
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2964 - loss: 1.3792 - val_accuracy: 0.2933 - v

In [38]:
loss, accuracy = model.evaluate(validation_sequences, validation_labels)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.2f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3059 - loss: 1.3769
Validation Loss: 1.3760
Validation Accuracy: 0.29
